# Emoji Prediction Project( A multinomial classification problem)


## 1. Installing the python emoji package 

In [64]:
!pip install emoji

In [65]:
import emoji 

In [66]:
emoji.EMOJI_UNICODE #Will show the dictionary of emojis

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_Is

Building up our own dictionary 

In [67]:
emoji_dict={"0":"\u2764\uFE0F",
            "1":":baseball:",
            "2":":grinning_face_with_big_eyes:",
            "3":":disappointed_face:",
            "4":":fork_and_knife:",
            "5":":hundred_points:",
            "6":":fire:",
            "7":":face_blowing_a_kiss:",
            "8":":chestnut:",
            "9":":flexed_biceps:"
}

In [68]:
#Check the dictionary:
for e in emoji_dict.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


## 2. Processing custom emoji dataset

In [69]:
import pandas as pd
import numpy as np

In [70]:
train=pd.read_csv('train_emoji.csv',header=None)
test=pd.read_csv('test_emoji.csv',header=None)

In [71]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [72]:
data=train.values

In [73]:
print(data.shape)

(132, 4)


In [74]:
X_train=train[0]
Y_train=train[1]
X_test=test[0]
Y_test=test[1]

In [75]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dict[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


## 3. Converting Sentences into Embeddings (using GloveVectors)

### Since the given dataset is small, we will make use of transfer learning in this project. We will use Glove 6B.50D glove vector embedding. It has pre trained word vectors with a vocab size of 6 Billion words.

In [76]:
f=open('glove.6B.50d.txt',encoding='utf-8')

In [77]:
#For fast lookup, we will build a dictionary of 6billion words and their corresponding vector
embeddings_index={}
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeddings_index[word]=coefs
f.close()

In [78]:
embeddings_index['cat']

array([ 0.45281 , -0.50108 , -0.53714 , -0.015697,  0.22191 ,  0.54602 ,
       -0.67301 , -0.6891  ,  0.63493 , -0.19726 ,  0.33685 ,  0.7735  ,
        0.90094 ,  0.38488 ,  0.38367 ,  0.2657  , -0.08057 ,  0.61089 ,
       -1.2894  , -0.22313 , -0.61578 ,  0.21697 ,  0.35614 ,  0.44499 ,
        0.60885 , -1.1633  , -1.1579  ,  0.36118 ,  0.10466 , -0.78325 ,
        1.4352  ,  0.18629 , -0.26112 ,  0.83275 , -0.23123 ,  0.32481 ,
        0.14485 , -0.44552 ,  0.33497 , -0.95946 , -0.097479,  0.48138 ,
       -0.43352 ,  0.69455 ,  0.91043 , -0.28173 ,  0.41637 , -1.2609  ,
        0.71278 ,  0.23782 ], dtype=float32)

## 4. Function for generating embedding layer output before feeding the RNN/LSTM Model 

In [79]:
def embedding_output(X):
    maxlen=10 #Maximum word in each sentence is 10
    embedding_out=np.zeros((X.shape[0],maxlen,50))
    
    for ix in range(X.shape[0]):
        X[ix]=X[ix].split() #Split sentences into list of words
        for ij in range(len(X[ix])):
            try:
                embedding_out[ix][ij]=embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij]=np.zeros((50,))
    return embedding_out

embeddings_matrix_train=embedding_output(X_train)
embeddings_matrix_test=embedding_output(X_test)


print(embeddings_matrix_train.shape)
print(embeddings_matrix_test.shape)

C:\Users\Pratyush Kumar\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(132, 10, 50)
(56, 10, 50)


## 5. Creating a LSTM Architecture


### We will build a simple sequential RNN Architecture with 10 cells. At output of RNN is a classifier with 5 nodes.
### Using stacked LSTM improves accuracy.

In [80]:
!pip install keras
from keras.models import Sequential
from keras.layers import *

In [81]:
model=Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_4 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [82]:
from keras.utils import to_categorical

In [83]:
Y_train=to_categorical(Y_train,num_classes=5)
Y_test=to_categorical(Y_test,num_classes=5)
print(Y_train.shape)
print(Y_train[0])

(132, 5)
[0. 0. 0. 1. 0.]


In [56]:
##from keras.callbacks import EarlyStopping
##from keras.callbacks import ModelCheckpoint

In [57]:
##Saving the best model - stop the training when overfitting
##checkpoint=ModelCheckpoint("bes_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
##earlystop=EarlyStopping(monitor='val_acc',patience=10)

In [84]:
hist=model.fit(embeddings_matrix_train,Y_train,epochs=100,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 4s 37ms/step - loss: 1.6208 - accuracy: 0.1714 - val_loss: 1.6097 - val_accuracy: 0.2222
Epoch 2/100
105/105 [==============================] - 0s 1000us/step - loss: 1.5599 - accuracy: 0.3333 - val_loss: 1.6200 - val_accuracy: 0.2222
Epoch 3/100
105/105 [==============================] - 0s 980us/step - loss: 1.5498 - accuracy: 0.3048 - val_loss: 1.6377 - val_accuracy: 0.2222
Epoch 4/100
105/105 [==============================] - 0s 1ms/step - loss: 1.5152 - accuracy: 0.3048 - val_loss: 1.6589 - val_accuracy: 0.2222
Epoch 5/100
105/105 [==============================] - 0s 774us/step - loss: 1.4671 - accuracy: 0.3619 - val_loss: 1.6826 - val_accuracy: 0.2222
Epoch 6/100
105/105 [==============================] - 0s 888us/step - loss: 1.4763 - accuracy: 0.3333 - val_loss: 1.6935 - val_accuracy: 0.2593
Epoch 7/100
105/105 [==============================] - 0s 712us/step - loss: 1.4501 - a

105/105 [==============================] - 0s 649us/step - loss: 0.0374 - accuracy: 1.0000 - val_loss: 1.1776 - val_accuracy: 0.6296
Epoch 58/100
105/105 [==============================] - 0s 534us/step - loss: 0.0382 - accuracy: 1.0000 - val_loss: 1.2301 - val_accuracy: 0.6296
Epoch 59/100
105/105 [==============================] - 0s 572us/step - loss: 0.0290 - accuracy: 1.0000 - val_loss: 1.1831 - val_accuracy: 0.6296
Epoch 60/100
105/105 [==============================] - 0s 706us/step - loss: 0.0207 - accuracy: 1.0000 - val_loss: 1.1750 - val_accuracy: 0.6296
Epoch 61/100
105/105 [==============================] - 0s 749us/step - loss: 0.0252 - accuracy: 1.0000 - val_loss: 1.2278 - val_accuracy: 0.5926
Epoch 62/100
105/105 [==============================] - 0s 689us/step - loss: 0.0524 - accuracy: 0.9810 - val_loss: 1.3919 - val_accuracy: 0.6296
Epoch 63/100
105/105 [==============================] - 0s 655us/step - loss: 0.0234 - accuracy: 1.0000 - val_loss: 1.6855 - val_accuracy

In [85]:
model.evaluate(embeddings_matrix_test,Y_test)

56/56 [==============================] - 0s 205us/step


[1.9310018164770943, 0.625]

## 6. Outputting Emojis for test data 

In [87]:

pred=model.predict_classes(embeddings_matrix_test)

In [90]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dict[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dict[str(pred[i])]))
    

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😞
she got me a present
❤️
😃
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
😞
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious ha ha
😃
😞
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😞
Congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
❤️
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
😃
Why are you feeling bad
😞
😞
I am upset
😞
😞
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
⚾
valentine day is near
❤️
😃
